---
title: 多水平验证性因子分析CFA理论和应用教程
date: 2024-03-18 11:27:59
tags: [mplus, sem, cfa, R]
mathjax: true
---

多水平验证性因子分析是一种在Mplus中用于分析复杂调查数据的方法。这种方法包括两个或更多水平的数据，每个水平代表一个不同的数据来源或测量层次。例如， 你的数据来自于多个学校的学生，学生是以学校为单位抽取得到的， 那么学生数据之间存在相关性。 比较适合多水平的验证性因子分析（Multilevel Confirmatory Factor Analysis，以下简称 MCFA 。

在纵向数据中， 一个个体会被测量多次， 我们将每个时间点的数据作为样本， 那么时间点是嵌套在个体之中的， 
这种结构的数据与前面的嵌套数据结构是一致的。


<!-- more -->

嵌套数据中， 我们有时候用到了潜变量进行建模， 比如做多水平的结构方程模型， 这时候需要提前做验证性因子分析。
所以本篇教程是后续多水平结构方程模型教程的预备课程。

## 什么是纵向数据

Longitudinal Data指的是在同一组个体或人群中，随着时间的推移追踪观察，记录同一组数据的情况。例如，一个研究小组跟踪一组年轻人的成长过程，记录他们的身高、体重、饮食、运动等数据随时间的变化。这种数据类型在心理学、医学、社会学等领域的研究中非常常见。

这种数据有很多存储的格式， 一种常见的格式被称为“长格式”， 例如：

| 受试者ID | 测量轮次 | 测量值 |  
| :--: | :--: | :--: |  
| 001 | 1 | 25.5 |  
| 001 | 2 | 26.2 |  
| 001 | 3 | 25.8 |  
| 001 | 4 | 26.1 |  
| 002 | 1 | 24.9 |  
| 002 | 2 | 25.3 |  
| 002 | 3 | 25.1 |  
| 002 | 4 | 25.5 |  
| ... | ... | ... |

## 案例介绍

<img src="imgs/multilevel-cfa.png">

在我们的简单案例中， 我们有两个潜变量， f1 和 f2， 每个潜变量有三个观测指标， 
总共6个观测指标， 分别是 y1-y6, 数据当中还有一个变量是ID， 代表每个被试的编码， 我们看下数据：


| y1  |    y2 |    y3 |    y4 |     y5 |     y6 |    ID|
| :--: | :--: | :--: | :--: | :--: | :--: | :--: |
| 0.081987|    -0.287225|    -2.033206|    -2.032697|     1.418202|     0.513686|    1|
|-0.928530|    -2.196663|     3.377093|     0.525700|     1.540415|     0.420246|    1|
|-0.884323|    -7.290047|    -0.425843|    -1.776992|    -2.493705|    -0.259916|    1|
| 0.901712|    -0.411791|    -2.056783|     1.971860|    -5.170934|     0.994531|    1|
|-2.199115|    -0.326509|    -2.277330|    -1.955370|    -0.591493|    -1.027715|    1|
| 0.000974|     3.046932|     0.469873|    -0.927001|    -0.266280|    -3.880203|    2|
|-1.448368|    -0.911395|     1.312358|     1.142734|     2.387918|    -3.926418|    2|

## 为什么必须用多水平CFA

普通的CFA要求样本是独立的， 但是我们现在的样本是纵向数据， 比如数据的前5行是一个被试的5个时间点的样本。
这违背了普通CFA的假定， 并且如果你强行做普通的CFA， 你的模型无法解释由于由于样本重复导致的过高的相关性。

普通的验证性因子分析模型如下：

以观测指标y1为例

$$
y1 = h_1 + \lambda_1 f_1 + \epsilon_1
$$

$f_1$是潜变量， 它通过等号右边的算式来预测观测指标y1， 对于所有的被试，在所有时间的观测样本， $h_1$是常数， 意思是不变的, 在上面的方程中起着截距的作用， 和回归上的截距是类似的，它代表了潜变量f1取0的时候y1的值。 但是因为是嵌套数据， $h_1$ 不变这个假定并不能反映真实情况， 很有可能  $h_1$ 因为被试的不同而不同， 即在潜变量f1取0的时候， 每个被试的观测值y1可能是不同的，并且同一个被试不同观测时间的截距可能是相同的。 所以我们可以假定  $h_1$ 是随个体变化的， 即  $h_{1i}$, i表示被试id。

$h_{1i}$是一个变量， 因此我们的模型是这样的：

$$
y1 = h_{1i} + \lambda_{1} f_1 + \epsilon_1
$$

但是这还不够， 既然$h_{1i}$是变量， 我们有6个观测指标，用j表示观测指标， 就会得到6个$h_{ji}$。 $h_{1i}-h_{3i}$由于是来自同一个因子下的观测指标， 所以我们有理由认为$h_{1i}-h_{3i}$背后也有可能是一个因子， 我们因此使用$fb1$表示他们背后的因子， 建模如下：

$$
h_{1i} = \gamma_{1i} fb_1 + \epsilon_1
$$

综上所述， 我们把上图中的模型都用数学表达式写出来就是：

$$
y1 = h_{1i} + \lambda_{1} fw_1 + \epsilon \\
y2 = h_{2i} + \lambda_{2} fw_1 + \epsilon \\
y3 = h_{3i} + \lambda_{3} fw_1 + \epsilon \\
y4 = h_{4i} + \lambda_{4} fw_2 + \epsilon \\
y5 = h_{5i} + \lambda_{5} fw_2 + \epsilon \\
y6 = h_{6i} + \lambda_{6} fw_2 + \epsilon \\
h_{1i} = \gamma_{1} fb_1 + \epsilon \\
h_{2i} = \gamma_{2} fb_1 + \epsilon \\
h_{3i} = \gamma_{3} fb_1 + \epsilon \\
h_{4i} = \gamma_{4} fb_2 + \epsilon \\
h_{5i} = \gamma_{5} fb_2 + \epsilon \\
h_{6i} = \gamma_{6} fb_2 + \epsilon \\
$$

为了简便残差没有标角标。

## 多层验证性因子分析如何用MPLUS实现

我们先看代码， 然后再解读。

```
TITLE: 两层两因子验证性因子分析案例
DATA: FILE IS ex9.11.dat;
VARIABLE: NAMES ARE y1-y6 id;
    CLUSTER = id;
ANALYSIS: TYPE = TWOLEVEL;
MODEL:
    %WITHIN%
    fw1 BY y1-y3;
    fw2 BY y4-y6;
    %BETWEEN%
    fb1 BY y1-y3;
    fb2 BY y4-y6;

OUTPUT:
    STANDARDIZED;
```

### TITLE

TITLE用于指定标题， 你可以写任意你想要的标题， 它不参与分析， 但是帮助你理解和整理结果

### DATA

DATA命令用于指定数据文件， 固定写法就是 $ FILE IS$ + 文件名称， 如果你的mplus脚本与数据文件不在一个文件夹，需要写完整的路径。

### VARIABLE

VARIABLE 用于设置数据中的变量名， `NAMES ARE`后面写变量名， 注意这个变量名的个数要跟数据文件中的列的个数相同。 确保每一列都有变量名。

CLUSTER 用于指定分组变量， 也就是说mplus需要根据这个变量确定你的样本是如何分组的， 比如我们的数据是一个被试连续5次测量， 那么被试ID就是一个分组变量。但是在其他数据结构里， 有可能是班级ID或者学校ID， 具体要根据你的研究设计来确定。

### ANALYSIS

ANALYSIS 用于设定分析的方法， TYPE 用于指定 TWOLEVEL ， 告诉mplus我们构建的是一个两层的模型。
有时候我们会在 ANALYSIS 设定估计器， 它是计算参数使用的方法， 在不同类型的模型里， 估计器的默认值是不同的，在多水平的模型中， 默认的估计器是 MLR（maximum likelihood with robust standard errors）。

如果你对统计不太精通，不建议自己设定估计器， 它是给精通统计和结构方程的专家提供的，在你不清楚具体算法的情况下， 更改估计器无法达到你想要的结果。

### MODEL

这部分是重点， 我们的模型分层模型， 所以在这部分可以分别定义%WITHIN%和%BETWEEN%两层模型。

%WITHIN% 是组内的变量关系， %BETWEEN% 是组间的变量关系， 在我们的数据中， 组内就是被试内部随时间变化的变量， 组间就是被试间不随时间变化的变量。

在 %WITHIN% 部分， 我们定义了随机截距模型， 但是没有任何代码告诉mplus我们使用了随机截距， 这是mplus默认的行为。 我们通过定义`fw1 BY y1-y3;`告诉mplus产生 `y1-y3`的随机截距，名字也叫做y1-y3， 这些随机截距是组间变量， 可以在 %BETWEEN% 部分使用。

%BETWEEN% 部分我们定义了两个潜变量， 有时候有些专家也会定义一个潜变量影响所有的随机截距。 潜变量的观测指标（也是椭圆）是  %WITHIN% 部分定义的随机截距。


## 代码的运行

我们使用R中的Mplusautomation这个库来自动化mplus， 如果你不会R也不用担心， 因为你可以自行在mplus中运行上面的mplus脚本。 为什么我用的是R呢， 因为 Mplusautomation 可以自动化处理很多繁琐的事情， 比如整理结果。

In [2]:
library(MplusAutomation)

ex9.11 <- read.table("ex9.11.dat", col.names = c("y1", "y2", "y3", "y4", "y5", "y6", "id"))

ex9.11_mobj <- mplusObject(
  TITLE="MultiLevel CFA",
  VARIABLE="CLUSTER = id;",
  ANALYSIS = "TYPE = TWOLEVEL;
    ESTIMATOR=MLR;",
  MODEL = "%WITHIN%
            fw1 BY y1-y3;
            fw2 BY y4-y6;
            %BETWEEN%
            fb1 BY y1-y3;
            fb2 BY y4-y6;",
  OUTPUT="STANDARDIZED;",
  rdata = ex9.11
)

ex9.11_fit <- mplusModeler(ex9.11_mobj, modelout = "ex9.11.inp", run=TRUE)

In [14]:
ex9.11_fit$results$summaries[ c("AICC","ChiSqM_Value", "ChiSqM_DF",  "CFI", "TLI","RMSEA_Estimate", "SRMR.Within", "SRMR.Between")]

AICC,ChiSqM_Value,ChiSqM_DF,CFI,TLI,RMSEA_Estimate,SRMR.Within,SRMR.Between
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
27956.52,13.75,16,1,1.006,0,0.014,0.062


## 区分效度

通常区分效度有很多证明的方法， 其中之一就是构建多个因子模型， 从单因子到多因子模型，
就本案例来说，因为我们有两个因子， 如果2因子模型在所有模型中是拟合度最好的，就可以证明我们的变量之间具有区分效度。

下面我们构建一个单因子的模型：

In [15]:
ex9.11_mobj2 <- mplusObject(
  TITLE="MultiLevel CFA",
  VARIABLE="CLUSTER = id;",
  ANALYSIS = "TYPE = TWOLEVEL;
    ESTIMATOR=MLR;",
  MODEL = "%WITHIN%
            fw1 BY y1-y6;
            %BETWEEN%
            fb1 BY y1-y6;",
  OUTPUT="STANDARDIZED;",
  rdata = ex9.11
)

ex9.11.single.factor_fit <- mplusModeler(ex9.11_mobj2, modelout = "ex9.11.single.factor.inp", run=TRUE)
ex9.11.single.factor_fit$results$summaries[ c("AICC","ChiSqM_Value", "ChiSqM_DF",  "CFI", "TLI","RMSEA_Estimate", "SRMR.Within", "SRMR.Between")]


AICC,ChiSqM_Value,ChiSqM_DF,CFI,TLI,RMSEA_Estimate,SRMR.Within,SRMR.Between
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
28053.53,149.483,18,0.804,0.673,0.085,0.06,0.119


将两个模型的拟合度整合到一起：

In [29]:
fit1 = ex9.11_fit$results$summaries[ c("AICC","ChiSqM_Value", "ChiSqM_DF",  "CFI", "TLI","RMSEA_Estimate", "SRMR.Within", "SRMR.Between")]
fit2 = ex9.11.single.factor_fit$results$summaries[ c("AICC","ChiSqM_Value", "ChiSqM_DF",  "CFI", "TLI","RMSEA_Estimate", "SRMR.Within", "SRMR.Between")]
d = rbind(fit1, fit2)
data.frame(d, row.names = c('两因子',  '单因子'))

,AICC,ChiSqM_Value,ChiSqM_DF,CFI,TLI,RMSEA_Estimate,SRMR.Within,SRMR.Between
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
两因子,27956.52,13.750,16,1.000,1.006,0.000,0.014,0.062
单因子,28053.53,149.483,18,0.804,0.673,0.085,0.060,0.119


## 参考文献

https://www.scirp.org/journal/paperinformation?paperid=92317